In [1]:
import pandas as pd
import numpy as np
import re

In [7]:
# Define the log file path
log_file = r"C:\Users\janbe\Documents\GitHub Repos\pulse\output_to_keep\results_benchmark\llms\grok4\zhu_2024b_zero_shot_preprocessor\log_20250721_112523.log"
output_path = r"C:\Users\janbe\Documents\GitHub Repos\pulse\output_to_keep\results_benchmark\llms\grok4\zhu_2024b_zero_shot_preprocessor\Grok4_sepsis_miiv_20250721_112523_metadata_reconstructed.csv"

In [8]:
# Prepare regex patterns for each field
block_start = re.compile(r"Decoded output:", re.MULTILINE)
decoded_output = re.compile(r"```json\s+({.*?})\s+```", re.DOTALL)
tokenization = re.compile(
    r"Tokenization time: ([\d.]+)s \| Inference time: ([\d.]+)s \| Input Tokens: (\d+) \| Output Tokens: (\d+)"
)
prob_label = re.compile(r"Predicted probability: ([\d.]+) \| True label: (\d+)")
diagnosis = re.compile(r"Diagnosis for: (.+)")
explanation = re.compile(r"Generated explanation: (.+)")
system_message = re.compile(r"System message: (.+)")
system_message_idx = re.compile(r"System message index: (\d+)")

# Read the log file
with open(log_file, "r", encoding="utf-8") as f:
    log = f.read()


# Only process after the start line

start_line = "Starting test evaluation..."

start_idx = log.find(start_line)

if start_idx != -1:

    log = log[start_idx + len(start_line) :]

In [9]:
# Split into blocks
blocks = re.split(block_start, log)
blocks = blocks[1:]  # Skip the first empty block
rows = []

In [10]:
for block in blocks:
    if not block.strip():
        continue

    # Extract fields
    decoded = decoded_output.search(block)
    diagnosis_json = None
    if decoded:
        import json

        try:
            diagnosis_json = json.loads(decoded.group(1))
        except Exception:
            diagnosis_json = {}

    token_match = tokenization.search(block)
    prob_label_match = prob_label.search(block)
    diagnosis_match = diagnosis.search(block)
    explanation_match = explanation.search(block)
    system_msg_match = system_message.search(block)
    system_msg_idx_match = system_message_idx.search(block)

    row = {
        "Input Prompt": "",  # Not present in your sample, fill as needed
        "Target Label": prob_label_match.group(2) if prob_label_match else "",
        "Predicted Probability": prob_label_match.group(1) if prob_label_match else "",
        "Predicted Diagnosis": (
            diagnosis_json.get("diagnosis", "") if diagnosis_json else ""
        ),
        "Predicted Explanation": (
            diagnosis_json.get("explanation", "") if diagnosis_json else ""
        ),
        "Tokenization Time": token_match.group(1) if token_match else "",
        "Inference Time": token_match.group(2) if token_match else "",
        "Input Tokens": token_match.group(3) if token_match else "",
        "Output Tokens": token_match.group(4) if token_match else "",
        "System Message": system_msg_match.group(1) if system_msg_match else "",
        "System Message Index": (
            system_msg_idx_match.group(1) if system_msg_idx_match else ""
        ),
    }
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(
    rows,
    columns=[
        "Input Prompt",
        "Target Label",
        "Predicted Probability",
        "Predicted Diagnosis",
        "Predicted Explanation",
        "Tokenization Time",
        "Inference Time",
        "Input Tokens",
        "Output Tokens",
        "System Message",
        "System Message Index",
    ],
)

In [11]:
df["System Message Index"] = np.tile(np.arange(5), len(df))[:len(df)]
df

,Input Prompt,Target Label,Predicted Probability,Predicted Diagnosis,Predicted Explanation,Tokenization Time,Inference Time,Input Tokens,Output Tokens,System Message,System Message Index
0,,0,0.8,,,,,,,,0
1,,0,0.25,,,,,,,,1
2,,0,0.75,,,,,,,,2
3,,0,0.18,,,,,,,,3
4,,0,0.2,,,,,,,,4
...,...,...,...,...,...,...,...,...,...,...,...
6170,,1,0.85,,,,,,,,0
6171,,1,0.8,,,,,,,,1
6172,,1,0.85,,,,,,,,2
6173,,1,0.85,,,,,,,,3


In [ ]:
df.to_csv(
    output_path,
    index=False,
)